# Laboratorio 03 Práctica 2 Julián Prieto Velasco, Javier Muñoz Rojas, Guillermo Ramírez Cárdenas, Pedro José Paniagua Falo

### Importación de Librerías

In [8]:
import pandas as pd
import sklearn
from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import LabelEncoder

### Lectura del CSV

In [9]:
datos = pd.read_csv('Recursos Lab3/Recursos Lab3/lluvias.csv')
datos.head(5)

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,51,42,True
1,1948-01-02,0.59,45,36,True
2,1948-01-03,0.42,45,35,True
3,1948-01-04,0.31,45,34,True
4,1948-01-05,0.17,45,32,True


### Funcion para ategorizar

In [10]:
# Definir una función para asignar una categoría a la variable "RAIN"
def asignar_categoria(lluvia):
    if lluvia == True:
        return "si"
    else:
        return "no"

### Preprocesamiento

In [11]:
# Definir las categorías para las temperaturas y las precipitaciones
temp_categorias = pd.cut(datos["TMIN"], bins=[-float("inf"), 10, 20, float("inf")], labels=["baja", "media", "alta"])
precip_categorias = pd.cut(datos["PRCP"], bins=[-float("inf"), 1, 2, float("inf")], labels=["poco", "normal", "mucho"])

# Crear nuevas columnas con las categorías
datos["TEMP_CAT"] = temp_categorias
datos["PRECIP_CAT"] = precip_categorias


# Aplicar la función a la variable "RAIN" y crear una nueva columna con las categorías
datos["LLUVIA_CAT"] = datos["RAIN"].apply(asignar_categoria)
le = LabelEncoder()
datos["TEMP_CAT_NUM"] = le.fit_transform(datos["TEMP_CAT"])
datos["PRECIP_CAT_NUM"] = le.fit_transform(datos["PRECIP_CAT"])
# Mostrar el dataframe con las nuevas columnas
datos.head(5)

,DATE,PRCP,TMAX,TMIN,RAIN,TEMP_CAT,PRECIP_CAT,LLUVIA_CAT,TEMP_CAT_NUM,PRECIP_CAT_NUM
0,1948-01-01,0.47,51,42,True,alta,poco,si,0,2
1,1948-01-02,0.59,45,36,True,alta,poco,si,0,2
2,1948-01-03,0.42,45,35,True,alta,poco,si,0,2
3,1948-01-04,0.31,45,34,True,alta,poco,si,0,2
4,1948-01-05,0.17,45,32,True,alta,poco,si,0,2


#### Explicación del preprocesamiento

El preprocesamiento se realiza para preparar los datos antes de utilizarlos en el modelo de clasificación. En este caso, se han aplicado diferentes pasos de preprocesamiento:

1- Se ha creado una nueva variable "LLUVIA_CAT" a partir de la variable "RAIN" para asignar una categoría a los días de lluvia (si/no).
2.- Se han creado nuevas variables "TEMP_CAT" y "PRECIP_CAT" a partir de las variables "TMIN" y "PRCP", respectivamente. Estas variables categorizan la temperatura y la cantidad de precipitación en tres niveles: baja, media y alta para la temperatura, y poco, normal y mucho para la precipitación.
3.- Se ha aplicado la función "LabelEncoder()" para convertir las variables categóricas en variables numéricas. Esto es necesario para que el modelo pueda trabajar con ellas.

En resumen, el preprocesamiento se realiza para transformar los datos a un formato que pueda ser utilizado por el modelo de clasificación. En este caso, se han convertido las variables categóricas a variables numéricas y se han creado variables adicionales que ayudan a categorizar los datos en diferentes niveles.





### Clasificador

In [12]:
# Seleccionar las variables de entrada y la variable de salida
X = datos[["TEMP_CAT_NUM", "PRECIP_CAT_NUM"]]
y = datos["LLUVIA_CAT"]

# Crear el clasificador Naive Bayes y entrenarlo con los datos
clf = CategoricalNB()
clf.fit(X, y)

# Calcular el error del clasificador
error = 1 - clf.score(X, y)
print(f"Error del clasificador: {error:.2f}")

Error del clasificador: 0.41


En este ejercicio, se utiliza el algoritmo CategoricalNB de scikit-learn para crear el clasificador Naive Bayes. Se seleccionan las variables de entrada ("TEMP_CAT" y "PRECIP_CAT") y la variable de salida ("LLUVIA_CAT") del dataset preprocesado. Luego se entrena el clasificador con los datos utilizando el método fit(). Finalmente, se calcula el error del clasificador utilizando el método score() y se muestra por pantalla.

Una vez que tenemos el clasificador entrenado y evaluado, podemos usarlo para predecir si lloverá en los próximos 3 días. Para ello, necesitamos obtener los datos meteorológicos de los próximos días. Podemos hacerlo utilizando alguna API de pronósticos del tiempo, o simplemente buscando los datos en una página web. en este caso hemos utilizado: https://www.timeanddate.com/weather/spain/madrid/ext.

Suponiendo que hoy es el 21 de marzo de 2023, y queremos saber si lloverá en los próximos 3 días. Podemos buscar los datos meteorológicos para los días 22, 23 y 24 de marzo en la página web indicada anteriormente, y guardarlos en un archivo CSV con las mismas columnas que el dataset preprocesado anterior. Luego podemos cargar los datos en un nuevo DataFrame y hacer las predicciones con el clasificador de la siguiente manera:

### ¿Por qué se ha creado así el clasificador?

Las variables elegidas para clasificar son la temperatura y la precipitación, ya que son dos factores importantes que pueden influir en si va a llover o no en un determinado día.

En particular, la temperatura se divide en tres categorías: baja, media y alta, mientras que la precipitación se divide en tres categorías también: poco, normal y mucho. Estas categorías se definieron en el código utilizando la función "pd.cut()", que agrupa los valores en un número finito de categorías discretas.

Además, se utilizó la función "LabelEncoder()" de la librería scikit-learn para codificar las variables categóricas a números enteros. Esto se hace porque muchos algoritmos de aprendizaje automático requieren que los datos de entrada sean numéricos en lugar de categóricos.

En resumen, se eligieron estas variables y categorías específicas para el clasificador Naive Bayes, ya que se consideraron las más relevantes para predecir si lloverá o no en un día determinado. Sin embargo, se podrían utilizar otras variables o categorías en función de los datos disponibles y de la hipótesis que se quiera probar.

In [20]:
# Cargar los datos del archivo CSV
datos_nuevos = pd.read_csv("Recursos Lab3/Recursos Lab3/Lluvias_reales.csv")

# Preprocesar los datos de la misma manera que el dataset original
temp_categorias = pd.cut(datos_nuevos["TMIN"], bins=[-float("inf"), 10, 20, float("inf")], labels=["baja", "media", "alta"])
precip_categorias = pd.cut(datos_nuevos["PRCP"], bins=[-float("inf"), 1, 2, float("inf")], labels=["poco", "normal", "mucho"])
datos_nuevos["TEMP_CAT"] = temp_categorias
datos_nuevos["PRECIP_CAT"] = precip_categorias

# Codificar las variables categóricas a números enteros
le_temp = LabelEncoder()
le_precip = LabelEncoder()

datos_nuevos["TEMP_CAT_NUM"] = le_temp.fit_transform(datos_nuevos["TEMP_CAT"])
datos_nuevos["PRECIP_CAT_NUM"] = le_precip.fit_transform(datos_nuevos["PRECIP_CAT"])

# Hacer las predicciones con el clasificador
X_nuevos = datos_nuevos[["TEMP_CAT_NUM", "PRECIP_CAT_NUM"]]
y_pred = clf.predict(X_nuevos)

# Mostrar las predicciones por pantalla
for i, lluvia in enumerate(y_pred):
    dia = datos_nuevos["DATE"][i]
    if lluvia == "si":
        print(f"El día {dia} de marzo lloverá.")
    else:
        print(f"El día {dia} de marzo no lloverá.")

El día 2023-03-22 de marzo lloverá.
El día 2023-03-23 de marzo lloverá.
El día 2023-03-24 de marzo lloverá.


### Conclusiones

En este caso, el modelo fue entrenado con datos históricos contenidos en el archivo 'lluvias.csv', y se utilizó para predecir si lloverá o no en los días especificados en el archivo 'Lluvias_reales.csv'. Es posible que el modelo no sea preciso en las predicciones de este archivo de prueba debido a diferencias en los datos históricos y actuales.

Además, los datos de lluvia en el archivo 'Lluvias_reales.csv' son todos cero (0), lo que indica que no hay lluvia en ninguno de los días. Es posible que el modelo haya sido entrenado con una cantidad insuficiente de datos históricos para poder hacer predicciones precisas sobre días sin lluvia.

En resumen, el modelo puede no ser preciso en la predicción de lluvia en los días especificados en el archivo 'Lluvias_reales.csv', y es posible que se necesite un modelo más complejo o un conjunto de datos históricos más grande para obtener predicciones más precisas.